<a href="https://colab.research.google.com/github/gwangjinjeong/Pytorch/blob/master/basic/03_Logistic_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# *0.* Preparation

In [29]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
torch.manual_seed(1)

# *1.* Logistic Regression

* Hypothesis   
$H(x) = \frac{1}{1+e^-W^TX} $

* Cost   
$ cost(W) = -\frac{1}{m} \sum y \log\left(H(x)\right) + (1-y) \left( \log(1-H(x) \right) $   
즉, H(x)가 실제 정답 y와 근사라면 cost는 0에 가깝고   
같지 않을수록 cost는 높아질것이다.

## 1.1 Make the Training Data

In [0]:
x_data = [[1, 2], [2, 3], [3, 1], [4, 3], [5, 3], [6, 2]] # 6,2로 이루어진 data
y_data = [[0], [0], [0], [1], [1], [1]]

In [0]:
x_train = torch.FloatTensor(x_data)
y_train = torch.FloatTensor(y_data)

In [6]:
print('e^1 equals: ', torch.exp(torch.FloatTensor([1])))

e^1 equals:  tensor([2.7183])


$H(x) = \frac{1}{1+e^{-W^TX}} $ => $ \frac{1}{1+e^{-X\bullet W}}$ => $ \frac{1}{1+e^{-X\bullet W+b}}$ 


In [0]:
W = torch.zeros((2,1), requires_grad=True)
b = torch.zeros(1, requires_grad = True)

```python
x_train.matmul(W) # 의 의미는 X*W를 의미한다.
```


In [0]:
hypothesis = 1 / (1 + torch.exp(-(x_train.matmul(W) + b)))

In [10]:
print(hypothesis)
print(hypothesis.shape)

tensor([[0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000]], grad_fn=<MulBackward0>)
torch.Size([6, 1])


이것을 Torch에서는 `torch.sigmoid()`라는 function으로 지원한다.

In [0]:
hypothesis  = torch.sigmoid(x_train.matmul(W) + b)

In [12]:
print(hypothesis)
print(hypothesis.shape)

tensor([[0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000]], grad_fn=<SigmoidBackward>)
torch.Size([6, 1])


## 1.2 Cost function

In [13]:
losses = -(y_train[0] * torch.log(hypothesis[0]) +
  (1 - y_train[0]) * torch.log(1 - hypothesis[0]))
# log p(X=0;w) = 1 - logP(X=1;W)
print(losses)

tensor([0.6931], grad_fn=<NegBackward>)


In [14]:
cost = losses.mean() # 점화식형태의 다 더하고 갯수만큼 나누는것이기 때문에 평균.
print(cost)

tensor(0.6931, grad_fn=<MeanBackward0>)


In [15]:
#이것은 위와같다.
F.binary_cross_entropy(hypothesis, y_train)

tensor(0.6931, grad_fn=<BinaryCrossEntropyBackward>)

In [30]:
# 모델 초기화
W = torch.zeros((2, 1), requires_grad=True)
b = torch.zeros(1, requires_grad=True)
# optimizer 설정
optimizer = optim.SGD([W, b], lr=1)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):

    # Cost 계산
    hypothesis = torch.sigmoid(x_train.matmul(W) + b) # or .mm or @
    cost = -(y_train * torch.log(hypothesis) + 
             (1 - y_train) * torch.log(1 - hypothesis)).mean()

    # cost로 H(x) 개선
    optimizer.zero_grad()  # 기존에 grad를 구한것이 있으면 초기화를 하겠다. 이걸 안하면 기존 grad를 더하게 된다.
    cost.backward()
    optimizer.step()

    # 100번마다 로그 출력
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))

Epoch    0/1000 Cost: 0.693147
Epoch  100/1000 Cost: 0.134722
Epoch  200/1000 Cost: 0.080643
Epoch  300/1000 Cost: 0.057900
Epoch  400/1000 Cost: 0.045300
Epoch  500/1000 Cost: 0.037261
Epoch  600/1000 Cost: 0.031673
Epoch  700/1000 Cost: 0.027556
Epoch  800/1000 Cost: 0.024394
Epoch  900/1000 Cost: 0.021888
Epoch 1000/1000 Cost: 0.019852


## 1.3 Evaluation

In [21]:
hypothesis = torch.sigmoid(x_train.matmul(W) + b)
print(hypothesis[:5])

tensor([[2.7648e-04],
        [3.1608e-02],
        [3.8977e-02],
        [9.5622e-01],
        [9.9823e-01]], grad_fn=<SliceBackward>)


In [22]:
prediction = hypothesis >= torch.FloatTensor([0.5]) # 0.5이상이 되면 1과 0으로 되게 만듬 => byteTensor
print(prediction[:5])

tensor([[False],
        [False],
        [False],
        [ True],
        [ True]])


In [23]:
print(prediction[:5])
print(y_train[:5])

tensor([[False],
        [False],
        [False],
        [ True],
        [ True]])
tensor([[0.],
        [0.],
        [0.],
        [1.],
        [1.]])


In [24]:
correct_prediction = prediction.float() == y_train
print(correct_prediction)

tensor([[True],
        [True],
        [True],
        [True],
        [True],
        [True]])


## 1.4 Higher Implementation

In [0]:
x_data = [[1, 2], [2, 3], [3, 1], [4, 3], [5, 3], [6, 2]] # 6,2로 이루어진 data
y_data = [[0], [0], [0], [1], [1], [1]]

In [0]:
x_train = torch.FloatTensor(x_data)
y_train = torch.FloatTensor(y_data)

In [0]:
class BinaryClassifier(nn.Module): #추상클래스를 상속받아서 나만의 class를 만든다.
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(2, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        return self.sigmoid(self.linear(x))

In [0]:
model = BinaryClassifier()

In [68]:
# optimizer 설정
optimizer = optim.SGD(model.parameters(), lr=1) # 처음 선언한 모델 내의 linear, sigmoid 변수를 가지고 올꺼야.

nb_epochs = 100
for epoch in range(nb_epochs + 1):

    # H(x) 계산
    hypothesis = model(x_train)

    # cost 계산
    cost = F.binary_cross_entropy(hypothesis, y_train)

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    # 20번마다 로그 출력
    if epoch % 10 == 0:
        prediction = hypothesis >= torch.FloatTensor([0.5])
        correct_prediction = prediction.float() == y_train
        accuracy = correct_prediction.sum().item() / len(correct_prediction)
        print('Epoch {:4d}/{} Cost: {:.6f} Accuracy {:2.2f}%'.format(
            epoch, nb_epochs, cost.item(), accuracy * 100,
        ))


Epoch    0/100 Cost: 1.106729 Accuracy 50.00%
Epoch   10/100 Cost: 1.712465 Accuracy 50.00%
Epoch   20/100 Cost: 0.429914 Accuracy 83.33%
Epoch   30/100 Cost: 0.366707 Accuracy 83.33%
Epoch   40/100 Cost: 0.309610 Accuracy 83.33%
Epoch   50/100 Cost: 0.259915 Accuracy 83.33%
Epoch   60/100 Cost: 0.214979 Accuracy 100.00%
Epoch   70/100 Cost: 0.177992 Accuracy 100.00%
Epoch   80/100 Cost: 0.155141 Accuracy 100.00%
Epoch   90/100 Cost: 0.142717 Accuracy 100.00%
Epoch  100/100 Cost: 0.133101 Accuracy 100.00%
